In [1]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
import keras



2022-12-24 23:49:36.476604: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-24 23:49:36.644359: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-24 23:49:36.666742: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-24 23:49:36.666755: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
train_dir = 'FER2013/train'
test_dir = 'FER2013/test'


In [3]:
print(len(os.listdir(train_dir)))
print(len(os.listdir(test_dir)))

7
7


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1.0/255.0, 
                           rotation_range=40,
                           width_shift_range=0.2,
                           height_shift_range=0.2,
                           horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1.0/255.0)

trainDatagen=train_datagen.flow_from_directory(train_dir,
                                 target_size=(48,48),
                                 batch_size=100,
                                 class_mode='categorical',
                                 color_mode='grayscale')
                                
valDatagen=val_datagen.flow_from_directory(test_dir,
                                      target_size=(48,48),
                                      batch_size=100,
                                      class_mode='categorical',
                                      color_mode='grayscale')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


['FER2013',
 'VideoAnalysis.ipynb',
 'CREMA-D',
 'features.csv',
 'Untitled-1.ipynb',
 'data_path.csv',
 '.ipynb_checkpoints',
 'TextAnalysis.ipynb']

In [4]:
model = keras.Sequential([
    layers.Conv2D(64, (3,3), activation="relu", input_shape=(48, 48, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),
                  
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(7, activation="softmax")
    ])

2022-12-24 23:50:27.572945: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 23:50:27.573213: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-24 23:50:27.573281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-24 23:50:27.573315: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-24 23:50:27.583368: W tensorflow/c

In [5]:
model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 21, 21, 64)       256       
 hNormalization)                                                 
                                                        

In [9]:
hist = model.fit(train_datagen, validation_data=val_datagen, epochs=150)

ValueError: Failed to find data adapter that can handle input: <class 'keras.preprocessing.image.ImageDataGenerator'>, <class 'NoneType'>